In [18]:
import requests
import xmltodict
import gzip
import pandas as pd

site_map = 'https://www.eventbrite.com/sitemap_xml/sitemap_index.xml'
response = requests.get(site_map)
data = xmltodict.parse(response.content)


In [26]:
maxSubsitemapsToCrawl = 100
breakcounter = 0
dfSitemap = pd.DataFrame({'loc': []})

for r in data["sitemapindex"]["sitemap"]:
    try: 
        print(r["loc"])
        resSingle = requests.get(r["loc"], stream=True)
        if resSingle.status_code == 200:
            if resSingle.headers['Content-Type'] == 'binary/octet-stream':
                resSingle.raw.decode_content = True
                resSingle = gzip.GzipFile(fileobj=resSingle.raw)
            else: 
                resSingle = resSingle.text

            rawSingle = xmltodict.parse(resSingle)
            dataSingle = [[rSingle["loc"]] for rSingle in rawSingle["urlset"]["url"]]
            dfSingle = pd.DataFrame(dataSingle, columns=["loc"])
            dfSitemap = pd.concat([dfSitemap,dfSingle])
            print(len(dfSitemap))
    except Exception as e:
        print("something went wrong at: " + r["loc"])
        print(e)

    breakcounter += 1
    if breakcounter == maxSubsitemapsToCrawl:
        break

https://www.eventbrite.com/sitemap_xml/top_search_pages00.xml.gz
50000
https://www.eventbrite.com/sitemap_xml/top_search_pages01.xml.gz
100000
https://www.eventbrite.com/sitemap_xml/top_search_pages02.xml.gz
150000
https://www.eventbrite.com/sitemap_xml/top_search_pages03.xml.gz
200000
https://www.eventbrite.com/sitemap_xml/top_search_pages04.xml.gz
250000
https://www.eventbrite.com/sitemap_xml/top_search_pages05.xml.gz
300000
https://www.eventbrite.com/sitemap_xml/top_search_pages06.xml.gz
350000
https://www.eventbrite.com/sitemap_xml/top_search_pages07.xml.gz
400000
https://www.eventbrite.com/sitemap_xml/top_search_pages08.xml.gz
450000
https://www.eventbrite.com/sitemap_xml/top_search_pages09.xml.gz
500000
https://www.eventbrite.com/sitemap_xml/top_search_pages10.xml.gz
550000
https://www.eventbrite.com/sitemap_xml/top_search_pages11.xml.gz
600000
https://www.eventbrite.com/sitemap_xml/top_search_pages12.xml.gz
650000
https://www.eventbrite.com/sitemap_xml/top_search_pages13.xml.gz


In [29]:
dfSitemap.head()

,loc
0,https://www.eventbrite.com/d/nj--willingboro/c...
1,https://www.eventbrite.com/d/ny--mastic-beach/...
2,https://www.eventbrite.com/d/canada--st-bernar...
3,https://www.eventbrite.com/d/united-kingdom--m...
4,https://www.eventbrite.com/d/spain--fuengirola...


In [30]:
from bs4 import BeautifulSoup

def scrape_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

In [40]:
def write_file(text, filename):
    with open(filename, 'a') as f:
        f.write(text)

for _,page in dfSitemap.iterrows():
        text = scrape_text(page['loc'])
        fname = page['loc'].replace('https://www.eventbrite.com/d/', '').replace('-', '_').replace('/', '_')
        write_file(text, f'data/{fname}.txt')